In [46]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
train=pd.read_table('../input/tweets-01/train_tweets.txt', header=None)

In [87]:
test=pd.read_table('../input/tweets-01/test_tweets_unlabeled.txt', header=None)

In [49]:
train

,0,1
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...
...,...,...
328190,4357,Steelbox Demonstrates Open Video Framework wit...
328191,4357,Small Businesses Rely on Sage to Help Them Rid...
328192,4357,TimeSight Systems™ Announces Next-Generation P...
328193,4357,Diebold Makes Its Leading Monitoring Solutions...


In [50]:
test

,0
0,Some people say that rappers don’t have feelin...
1,Do you know how to tweet on a Blackberry 8830?...
2,Yoga is the cessation of mind. -Patanjali
3,"@handle Well, with my millions of dollars, a f..."
4,Cambria hotels free guide http://hotels.izigot...
...,...
35059,NCAA Football Odds - West Virginia at Cincinna...
35060,@handle My favorite color!! Congrats!
35061,It's not a simple matter to let others know wh...
35062,"Funny. The ""Smoke N Drank"" track off of In a M..."


In [51]:
def preprocess(df1):
    if df1.shape[1]==2:
        p=1
    if df1.shape[1]==1:
        p=0
        
        
        
    """ This is to remove @handle from the data."""
    lst=[]
    for i,k in enumerate(df1[p]):
        lst.append(k.replace('@handle',''))
    df1[p]=lst

    '''-----------------------------------------'''        
        
    '''-----Removing special characters---------'''
    lst=[]
    for i in df1[p]:
        letters_only = re.sub("[^a-zA-Z]"," ",i)
        lst.append(letters_only)
    df1[p]=lst
    '''-----------------------------------------'''
        
    '''----removing all the extra spaces--------'''
    df1[p]=df1[p].str.split()
    lst=[]
    for i in df1[p]:
        sentence = ' '.join(i)
        lst.append(sentence)
    df1[p]=lst
    df1[p]=df1[p].str.lower()
    '''-----------------------------------------'''
        
        
    '''only for training data'''
    if p==1:
        print(df1[df1[p]==''])   
        print('So removing these empty rows....')
        df1[p].replace('', np.nan,inplace=True)
        print(df1[p].isnull().sum())
        print('Results of null values in data')
        df1.dropna(axis=0, inplace=True)
        df1.reset_index(inplace=True)
        df1.pop('index')
            
        '''--------Removing re-tweets-----------'''
        lst=[]
        for i,k in enumerate(df1[p]):
            if k.split()[0]=='rt':
                lst.append(i)
                #train.drop(i, axis=0, inplace=True)
            
        '''For checking the presences of tweets more than one time.'''
        rt=df1.loc[lst,1] 
        c=0
        for i in rt:
            for j in rt:
                if j==i:
                    c=c+1
            if c==2:
                d=i
                break;
            c =0
        c=0
        for i in rt:
            if i=='rt plasma tv review panasonic th px u http aclnk com ar':
                c=c+1
        print(c)
        print('This shows number of rt is repeating more than once so it should be dropped.')
            
        df1.drop(lst, axis=0, inplace=True)
        print('Dropped!')
        '''----------------------------------------'''
        '''----------------------------------------'''
            
        df1.reset_index(inplace=True)
        df1.drop([ 'index'], axis=1, inplace=True)
            
    return df1

In [52]:
train_n= preprocess(train)

           0 1 
4010    8573   
4036    8573   
4263    8901   
4660    4375   
4664    4375   
...      ... ..
324170  8873   
325139  5106   
327422  7299   
327712  8045   
328165  1319   

[535 rows x 2 columns]
So removing these empty rows....
535
Results of null values in data
2
This shows number of rt is repeating more than once so it should be dropped.
Dropped!


In [53]:
test_n =preprocess(test)

In [55]:
train.iloc[18,1]

'only'

In [59]:
def remove_stopwords(train_n):
    if train_n.shape[1]==2:
        p=1
    if train_n.shape[1]==1:
        p=0
    stop = stopwords.words("english")
    train_n[p] = train_n[p].str.split()
    def removestopwords(y):   
        stopwordremoved =[w for w in y if not w in stop]
        return(" ".join(stopwordremoved))
    column_size = train_n.shape[0]

    cleaned1 = []
    for i in range( 0, column_size):
        cleaned1.append(removestopwords(train_n[p][i]))
    train_n[p] = cleaned1
    if p==1:
        train_n[p].replace('', np.nan,inplace=True)
        train_n.dropna(axis=0, inplace=True)
        train_n.reset_index(inplace=True)
        train_n.pop('index')
    if p==0:
        train_n[p].replace('', 'null',inplace=True)
    return train_n

In [60]:
train_new=remove_stopwords(train_n)

In [61]:
test_new =remove_stopwords(test_n)

In [62]:
train_new

,0,1
0,8746,let try catch live next week
1,8746,going watch grey big screen thursday indulgence
2,8746,pleasure patrick hope well
3,8746,hi traveling lot lots come next month recovere...
4,8746,u get invite justin
...,...,...
304250,4357,steelbox demonstrates open video framework sri...
304251,4357,small businesses rely sage help ride recession...
304252,4357,timesight systems announces next generation pl...
304253,4357,diebold makes leading monitoring solutions ava...


In [64]:
d={}
for i in train_n[1]:
  k=i.split()
  for m in k:
    if m in d.keys():
      d[m]=d[m]+1
    if m not in d.keys():
      d[m]=1

In [65]:
d

{'let': 3568,
 'try': 1837,
 'catch': 569,
 'live': 3626,
 'next': 3383,
 'week': 4174,
 'going': 6243,
 'watch': 3085,
 'grey': 131,
 'big': 2881,
 'screen': 259,
 'thursday': 702,
 'indulgence': 2,
 'pleasure': 159,
 'patrick': 273,
 'hope': 2916,
 'well': 3801,
 'hi': 961,
 'traveling': 113,
 'lot': 1424,
 'lots': 800,
 'come': 3699,
 'month': 990,
 'recovered': 27,
 'pneumonia': 64,
 'r': 3748,
 'u': 10938,
 'get': 13393,
 'invite': 313,
 'justin': 189,
 'think': 5618,
 'still': 4287,
 'good': 11096,
 'friend': 1407,
 'lol': 8827,
 'remember': 981,
 'fine': 482,
 'new': 15658,
 'great': 7754,
 'coach': 383,
 'want': 5248,
 'picture': 532,
 'sitting': 556,
 'understand': 415,
 'thanks': 7367,
 'rts': 253,
 'womma': 25,
 'summit': 224,
 'grrr': 52,
 'must': 1555,
 'crazy': 1175,
 'catching': 148,
 'trip': 707,
 'news': 3503,
 'dale': 45,
 'car': 1406,
 'jump': 226,
 'minutes': 937,
 'eta': 13,
 'wish': 1533,
 'could': 2696,
 'w': 5146,
 'stu': 14,
 'family': 1589,
 'drive': 749,
 'ho

In [66]:
len(d)

171725

In [42]:
max(d.values())

111480

In [67]:
nd=sorted(d, key=d.get)

In [68]:
nd[-100:-1]

['music',
 'never',
 'pm',
 'fun',
 'year',
 'looking',
 'gd',
 'look',
 'j',
 'business',
 'big',
 'follow',
 'hope',
 'post',
 'e',
 'th',
 'world',
 'watch',
 'help',
 'thank',
 'via',
 'say',
 'better',
 'tomorrow',
 'f',
 'morning',
 'man',
 'c',
 'take',
 'getting',
 'next',
 'game',
 'oh',
 'happy',
 'news',
 'let',
 'live',
 'life',
 'come',
 'r',
 'first',
 'well',
 'b',
 'much',
 'way',
 'ow',
 'n',
 'week',
 'blog',
 'still',
 'last',
 'best',
 'show',
 'home',
 'p',
 'would',
 'right',
 'im',
 'really',
 'twitpic',
 'video',
 'tonight',
 'night',
 'people',
 'w',
 'want',
 'work',
 'free',
 'make',
 'need',
 'think',
 'check',
 'back',
 'going',
 'us',
 'see',
 'got',
 'twitter',
 'thanks',
 'great',
 'know',
 'go',
 'rt',
 'tinyurl',
 'love',
 'today',
 'time',
 'lol',
 'one',
 'www',
 'day',
 'u',
 'good',
 'like',
 'get',
 'new',
 'com',
 'bit',
 'ly']

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [70]:
vec=TfidfVectorizer()

In [71]:
from sklearn.model_selection import train_test_split

In [88]:
X=train_new[1].to_list()
y=train_new[0].to_list()
t=test_new[0].to_list()

In [78]:
bow=vec.fit_transform(X)

In [89]:
bow_test=vec.transform(t)

In [0]:
'''from sklearn.neural_network import MLPClassifier

In [0]:
'''neu=MLPClassifier(hidden_layer_sizes=(100, ), 
             activation='relu', solver='adam', 
             alpha=0.0001, batch_size=8, 
             learning_rate='adaptive', 
             learning_rate_init=0.001, 
             power_t=0.5, max_iter=200, 
             shuffle=True, random_state=None, 
             tol=0.0001, verbose=False, 
             warm_start=False, momentum=0.9, 
             nesterovs_momentum=True, early_stopping=True,
             validation_fraction=0.2, beta_1=0.9,
             beta_2=0.999, epsilon=1e-08,
             n_iter_no_change=10, max_fun=15000)

In [0]:
'''neu.fit(bow, y_train)

In [0]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model = model.fit(bow, y)
pred=model.predict(bow_test)

In [ ]:
Answer=pd.DataFrame(pred)
Answer.to_csv('Answer.csv')